In [2]:
import pandas as pd 
jobs = pd.read_csv("C:\\Users\\jcars\\OneDrive\\HII_internshipProject\\Fleet-Sustainment-HVAC-Project\\RawData\\cvn-hvac-jobs-data.csv")
supply = pd.read_csv("C:\\Users\\jcars\\OneDrive\\HII_internshipProject\\Fleet-Sustainment-HVAC-Project\\RawData\\cvn-hvac-supply-data.csv")

jobs

C:\Users\jcars\AppData\Local\Temp\ipykernel_4156\2692723872.py:2: DtypeWarning: Columns (3,4,20) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs = pd.read_csv("C:\\Users\\jcars\\OneDrive\\HII_internshipProject\\Fleet-Sustainment-HVAC-Project\\RawData\\cvn-hvac-jobs-data.csv")


,uic,job_seq,action_taken_code,alt_closing,alt_opening,apl,cause_code,cdm_rin,csmp_narrative_summary,date_closing,...,when_discovered,cause,deferral_reason,feasible_with_ship_inventory,feasible_with_additional_dla_inventory,ship_readiness_effect,assigned,casrep_filed,ship,shipname
0,R03369,36493414,1,NaN,NaN,43A030026,7,86KYW,PARTS MISSING,5/8/2018,...,During operability tests,Normal wear and tear,NaN,False,False,NaN,False,False,CVN69,USS DWIGHT D. EISENHOWER
1,R03369,36493441,1,NaN,NaN,611330020,7,0KYWX,PARTS MISSING,5/10/2018,...,During operability tests,Normal wear and tear,NaN,False,False,NaN,False,False,CVN69,USS DWIGHT D. EISENHOWER
2,R23170,36496418,1,NaN,NaN,611330020,7,0UXWY,PARTS MISSING,5/29/2018,...,During operability tests,Normal wear and tear,NaN,False,False,NaN,False,False,CVN77,USS GEORGE H.W. BUSH
3,R21853,36305764,1,NaN,NaN,18550014,7,02THE,PARTS BROKEN,4/12/2018,...,During operability tests,Normal wear and tear,NaN,False,False,NaN,False,False,CVN75,USS HARRY S. TRUMAN
4,R23170,36962865,1,NaN,NaN,213450019,7,0QGUV,DAMPER WILL NOT RESET,11/29/2018,...,During operability tests,Normal wear and tear,NaN,False,False,NaN,False,False,CVN77,USS GEORGE H.W. BUSH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47345,R21297,41094097,NaN,NaN,NaN,48A030009,1,0KWUJ,SF FY23 PIA CLEAN VENT SYSTEM,NaN,...,During PMS,Abnormal environment,For ship's force overhaul or availability work...,False,False,0.0,False,False,CVN72,USS ABRAHAM LINCOLN
47346,R21247,41100681,NaN,NaN,NaN,XSYSTEMITEM,7,044VL,VENTILATION DETERIORATED,NaN,...,During inspection,Normal wear and tear,Lack of material,False,False,NaN,False,False,CVN71,USS THEODORE ROOSEVELT
47347,R21297,36922139,4,NaN,NaN,000A5120,1,0NVZZ,FAULTY MIXER,11/28/2018,...,Normal operation,Abnormal environment,Lack of material,False,False,NaN,False,False,CVN72,USS ABRAHAM LINCOLN
47348,R23173,41109742,NaN,NaN,NaN,32A150002,3,0A6PS,PLATE BENT,NaN,...,During PMS,Lack of knowledge or skill,Lack of facilities/capabilities,False,False,0.0,False,False,CVN78,USS GERALD R FORD


In [3]:
### Proprotions of frequencies table
jobs['apl'].value_counts(normalize = 'True').round(2) #Proportions using normalize 



apl_lengths = [len(s) for s in jobs['apl'].astype('string').dropna()]  #Table displays how often an apl has 8 characters, 9 characters, etc. 
df = pd.DataFrame({'length':apl_lengths}) 
table = df['length'].value_counts().reset_index()
table 

,length,count
0,9,30663
1,11,7803
2,8,4950
3,10,3227
4,5,368
5,4,251
6,7,2


In [ ]:
#looking at components of location
sub = jobs[jobs['location'].str.count('-') == 3]
components = sub['location'].str.split("-", expand = True)
components = components.rename(columns = {0: 'Level', 1: 'Front to Back', 2: 'Distance from Center', 3: 'Purpose'})


# components['Level'].value_counts(normalize=True).round(2)
# components['Front to Back'].value_counts(normalize=True).round(2)
# # components['Distance from Center'].value_counts(normalize=True).round(2)
# # components['Purpose'].value_counts(normalize=True).round(2)